In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
#try:
#    os.chdir("L:\laupodteam\AIOS\Huibert-Jan\Celldynclustering\celldyn_embedder")
#except FileNotFoundError:
#    os.chdir('C:/Users/Huibert-Jan/Documents/Werk/UMCU/celldyn_embedder')    

import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pacmap
import umap
import trimap

#from scipy.stats import chisquare, chi2_contingency, pearsonr
#from scipy.stats import kendalltau,spearmanr, weightedtau, theilslopes, wilcoxon, ttest_rel
#from scipy.spatial import distance
#import dcor

from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#from sklearn.decomposition import PCA
#from sklearn.manifold import Isomap, MDS, SpectralEmbedding
from sklearn.manifold import LocallyLinearEmbedding as LLE, TSNE, smacof, trustworthiness
from sklearn.cluster import KMeans, BisectingKMeans, OPTICS, AffinityPropagation, AgglomerativeClustering, SpectralClustering
# add GMM
from sklearn.mixture import GaussianMixture as GMM, BayesianGaussianMixture as BGMM

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

#import skfuzzy as fuzz


#from sklearn.metrics import rand_score, adjusted_mutual_info_score, adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
#from sklearn.neighbors import KNeighborsClassifier as KNN
#from sklearn.cluster import KMeans

from hembedder.utils.distance import poincarre_dist, hyperboloid_dist, fractional_distance, Distance
from hembedder.utils.quality_metrics import CDEmbeddingPerformance
#import numpy.linalg as la
#import torch 

#from numba import njit

#import faiss
import gc

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from scipy.stats import entropy

from tqdm import tqdm

from pysr import PySRRegressor
from gplearn.genetic import SymbolicTransformer
#from sklego.meta import ZeroInflatedRegressor
#from lineartree import LinearTreeRegressor, LinearForestRegressor

from xgboost import XGBRegressor, XGBRFClassifier

from typing import List, Tuple, Iterable, Callable, Literal


import matplotlib.colors as mcolors
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI

import random


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


d:\VENVS\Envs\hema_embedder\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
# take class BGMM and extend with a function called bic
class BGMM(BGMM):
    def _n_parameters(self, X):
          """Return the number of free parameters in the model."""
          _, n_features = self.means_.shape
          # Number of effective components equals the number of unique labels
          n_effect_comp = len(np.unique(self.predict(X)))
          #n_effect_comp = self.n_components
          if self.covariance_type == 'full':
              cov_params = n_effect_comp * n_features * (n_features + 1) / 2.
          elif self.covariance_type == 'diag':
              cov_params = n_effect_comp * n_features
          elif self.covariance_type == 'tied':
              cov_params = n_features * (n_features + 1) / 2.
          elif self.covariance_type == 'spherical':
              cov_params = n_effect_comp
          mean_params = n_features * n_effect_comp
          return int(cov_params + mean_params + n_effect_comp - 1)

    def bic(self, X):
        """Bayesian information criterion for the current model on the input X.

        Parameters
        ----------
        X : array of shape (n_samples, n_dimensions)

        Returns
        -------
        bic : float
            The lower the better.
        """
        return (-2 * self.score(X) * X.shape[0] + self._n_parameters(X) * np.log(X.shape[0]))

    def aic(self, X):
        """Akaike information criterion for the current model on the input X.

        Parameters
        ----------
        X : array of shape (n_samples, n_dimensions)
+
        Returns
        -------
        aic : float
            The lower the better.
        """
        return -2 * self.score(X) * X.shape[0] + 2 * self._n_parameters(X)


In [5]:
def cluster_classification_f1(X, labels):
    '''
        create a logistic regression classifier to predict cluster labels
        , determine the stratified cross-validated F1 score of the model.
    '''
    # create a logistic regression classifier to predict cluster labels
    clf = LogisticRegression(random_state=0, penalty='elasticnet', solver='saga', l1_ratio=0.1,
                                multi_class='multinomial', max_iter=5000)

    # determine the stratified cross-validated F1 score of the model.
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    f1_scores = []
    for train_index, test_index in skf.split(X, labels):
        clf.fit(X[train_index], labels[train_index])
        f1_scores.append(f1_score(labels[test_index], clf.predict(X[test_index]), average='weighted'))

    return np.mean(f1_scores), np.std(f1_scores)

In [6]:
# a function to run a clustering algorithm in a bootstrapping fashion
# the outcome is a list of clustering models
def clustering_bootstrapping(data: pd.DataFrame, 
                             n_clusters: int=10,
                             n_bootstraps: int=100, 
                             sample_size: int=50000,
                             clusterer: str='kmeans',
                             align: bool=False,
                             **kwargs)-> List[Callable]:
    if kwargs.get('n_init', None) is None:
        n_init = 10
    else:
        n_init = kwargs.get('n_init')
    
    model_list = []
    for i in tqdm(range(n_bootstraps)):
        # sample data
        sample = data.sample(sample_size, replace=True)
        # cluster sample
        if clusterer == 'kmeans':
            if (i==0) | (align==False):
                model = KMeans(n_clusters=n_clusters, random_state=123, n_init=n_init)
                model.fit(sample)
                init_means = model.cluster_centers_
            else:
                model = KMeans(n_clusters=n_clusters, random_state=123, init=init_means, n_init=n_init)
                model.fit(sample)
        elif clusterer == 'optics':
            model = OPTICS(min_samples=100, n_jobs=-1)
            model.fit(sample)
        elif clusterer == 'hierarchical':
            model = AgglomerativeClustering(n_clusters=n_clusters)
            model.fit(sample)
        elif clusterer == 'gmm':
            if (i==0) | (align==False):
                model = GMM(n_components=n_clusters, random_state=123)
                model.fit(sample)
                init_means = model.means_
            else:
                model = GMM(n_components=n_clusters, random_state=123, means_init=init_means)
                model.fit(sample)
        elif clusterer == 'bgmm':
            if (i==0) | (align==False):
                model = BGMM(n_components=n_clusters, random_state=123, max_iter=500)
                model.fit(sample)
                init_means = model.mean_prior_
            else:
                model = BGMM(n_components=n_clusters, random_state=123, mean_prior=init_means, max_iter=500)
                model.fit(sample)
        else:
            raise ValueError("Unknown clustering algorithm")
        model_list.append(model)
    return model_list
    
def assign_clusters(data: pd.DataFrame, 
                    model_list: List[Callable], 
                    clusterer='kmeans') -> pd.DataFrame:
    
    # get cluster labels
    if clusterer in ['kmeans']:
        labels = np.array([model.predict(data) for model in model_list])
        # get cluster centers
        centers = np.array([model.cluster_centers_ for model in model_list])
        # get cluster sizes
        sizes = np.array([np.bincount(label, minlength=centers.shape[1]) for label in labels])
    elif clusterer in ['gmm', 'bgmm']:
        labels = np.array([model.predict(data) for model in model_list])
        # get cluster centers
        centers = np.array([model.means_ for model in model_list])
        # get cluster sizes
        sizes = np.array([np.bincount(label, minlength=centers.shape[1]) for label in labels])        
    return labels, centers, sizes 


def distance(v1, v2, metric='euclidean'):
    ''' Calculate the distance between two vectors '''
    if metric=='euclidean':
        return np.linalg.norm(v1 - v2)
    elif metric=='cosine':
        return 1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    elif metric=='poincarre':
        return poincarre_dist(v1, v2)
    elif metric=='manhattan':
        return np.linalg.norm(v1 - v2, ord=1)

In [7]:
cd_path = 'L:/laupodteam/AIOS/Bram/data/CellDyn/artifacts/celldyn_FULL_transformed_df_updated.feather'
celldyn_full = pd.read_feather(cd_path)

meas_columns = [c for c in celldyn_full.columns if ('c_b' in c) | ("COMBO" in c)]
mode_columns = [c for c in celldyn_full.columns if 'c_m' in c]
alrt_columns = [c for c in celldyn_full.columns if 'alrt' in c.lower()]
c_s_columns = [c for c in celldyn_full.columns if 'c_s_' in c.lower()]
celldyn_full.drop(['index'], axis=1, inplace=True)

celldyn_full = celldyn_full.assign(gender=celldyn_full.gender.map({'M':0, 'F':1}))
celldyn_full.dropna(subset=['gender','draw_hour'], axis=0, inplace=True)
celldyn_full.rename(columns={'studyid_alle_celldyn':'study_id', 'afname_dt': 'sample_dt'}, inplace=True)
celldyn_full.set_index(['study_id', 'sample_dt'], inplace=True)

In [ ]:
celldyn_emb = pd.read_feather("L:\laupodteam\AIOS\Bram\data/CellDyn/artifacts/umap_euclidean_euclidean_spectral_dims_6_n_n_50_n_epochs_400_densmap_True_embedded_data.feather")
celldyn_emb = celldyn_emb.assign(sex=celldyn_emb.sex.map({'M':0, 'F':1}))
celldyn_emb.dropna(subset=['sex','draw_hour'], axis=0, inplace=True)
celldyn_emb.set_index(['study_id', 'sample_dt'], inplace=True)
celldyn_emb = celldyn_emb.loc[celldyn_full.index]

In [ ]:
in_full = celldyn_full.reset_index()[['study_id', 'sample_dt']].drop_duplicates()
in_emb = celldyn_emb.reset_index()[['study_id', 'sample_dt']].drop_duplicates()

## Multi-collinearity removal

In [8]:
tmp = celldyn_full[meas_columns].sample(1000).T
stnd = PowerTransformer(method='yeo-johnson', standardize=True)
clst = OPTICS(min_samples=5, leaf_size=10, metric="manhattan", cluster_method="xi", n_jobs=4)
#clst = AffinityPropagation(damping=0.5)
le_pipe = Pipeline([('stnd', stnd), 
                    ('clst', clst)], 
                    verbose=True)
clst.fit(tmp)

AttributeError: 'NoneType' object has no attribute 'split'

## Feature selection using cluster classifier-based model-selection

i.e. what features are important in seperating patient clusters


In [ ]:
# get cluster label

# get statistic test of difference between clusters for each feature

## Feature expansion using  symbolic regression

## Blob metrics

* We use the density profiles from HDBSCAN to characterize the blobbiness of the data
* We use fuzzy c-means in combination with AUFPC curves

In [ ]:
from hdbscan import HDBSCAN

sample_size = 25_000
selection = in_full.sample(sample_size).reset_index(drop=True)

kwargs = {
    'min_cluster_size': 5, 
    'min_samples': 15, 
    'metric': 'manhattan',
    'cluster_selection_method':'leaf'    
}

emb_index = celldyn_emb[celldyn_emb.index.isin(selection.itertuples(index=False))].index.drop_duplicates()
full_index = celldyn_full[celldyn_full.index.isin(selection.itertuples(index=False))].index.drop_duplicates()


In [ ]:
df = celldyn_full.loc[full_index].reset_index().groupby(['study_id', 'sample_dt']).first()
df_ = df[meas_columns].copy()

In [ ]:
Ranker = FunctionTransformer(lambda x: np.argsort(np.argsort(x, axis=0), axis=0))

class PreTrainedTransformer:
    def __init__(self, scaler):
        self.scaler = scaler  
        try:
            self.labels_ = scaler.labels_
        except:
            pass

    def transform(self, X):
        try:
            return self.scaler.transform(X)
        except:
            return self.scaler.predict(X)
    
    def fit(self, X, y=None):
        return self
    

def pipe_fitter_fixed(df, 
                scaler: Literal = ['standard', 'minmax', 'power'],
                clusterer: Literal = ['kmeans', 'spectral', 'agglomerative'],
                embedder: Literal = ['pca', 'umap', 'trimap', 'pacmap', 'RAW'],
                pre_ranking: bool = True,
                n_components: int = 6,
                n_clusters: int = 10, 
                pipe=None,
                pretrained=None,
                cluster_init='k-means++'
                ):
    '''
        in:
            df: dataframe with measurements
            clusterer: clusterer to use
            embedder: embedder to use
            pre_ranking: whether or not to use pre-ranking
            n_components: number of components to use for embedding
            n_clusters: number of clusters to use for clustering
        returns fitted pipeline
    '''
    assert((isinstance(cluster_init, str)) | (isinstance(cluster_init, np.ndarray)) | (cluster_init is None)), \
    f'cluster_init should be either a string or a numpy array, is {type(cluster_init)}'
    if isinstance(cluster_init, str):
        assert(cluster_init in ['k-means++', 'random'])
    if cluster_init is None:
        cluster_init = 'k-means++'
    
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    elif scaler == 'power':
        scaler = PowerTransformer()
        
    ######################################################    
    
    if (pipe is not None) & (pretrained=='clusterer'):
        clusterer = PreTrainedTransformer(pipe.named_steps['clusterer'])
    else:
        if clusterer == 'spectral':
            clusterer = SpectralClustering(n_clusters=n_clusters, random_state=123, n_init=10, affinity='rbf', n_neighbors=500)
        elif clusterer == 'kmeans':
            clusterer = KMeans(n_clusters=n_clusters, random_state=123, n_init=25, max_iter=1000, init=cluster_init)
        elif clusterer == 'agglomerative':
            clusterer = AgglomerativeClustering(n_clusters=n_clusters, linkage='average', metric='manhattan')

    ######################################################

    if (pipe is not None) & (pretrained=='embedder'):
        embedder = PreTrainedTransformer(pipe.named_steps['embedder'])
    else:
        if embedder == 'pca':
            embedder = PCA(n_components=n_components, whiten=False)
        elif embedder == 'umap':
            embedder = umap.UMAP(n_components=n_components, n_neighbors=500, n_epochs=500,
                                metric='manhattan', init='spectral',
                                random_state=123, densmap=False, min_dist=0)
        elif embedder == 'trimap':
            pipeline_emb = clusterer.fit(
                            trimap.TRIMAP(n_dims=n_components, n_inliers=12, n_outliers=4, 
                                        distance='manhattan', n_iters=500, lr=0.09).fit_transform(
                                scaler.fit_transform(df)
                                )
                            )
            return pipeline_emb
        elif embedder == 'pacmap':
            embedder = pacmap.PaCMAP(n_components=n_components, 
                                        n_neighbors=500, num_iters=500, lr=0.9, 
                                        apply_pca=False)
        elif embedder == 'RAW':
            embedder = None
    
    ######################################################
    
    if pre_ranking:
        scaler = None
        ranker = Ranker
    
    le_pipe_list = [('scaler', scaler)]
    le_pipe_list += [('ranker', ranker)] if pre_ranking else []
    le_pipe_list += [('embedder', embedder)]
    le_pipe_list += [('clusterer', clusterer)]    
    le_pipe = Pipeline(le_pipe_list)
    le_pipe.fit(df)
    
    return le_pipe
    
def pipe_fitter_density(df, 
                scaler: Literal = ['standard', 'minmax', 'power'],
                clusterer: Literal = ['hdbscan', 'optics'],
                embedder: Literal = ['pca', 'umap', 'trimap', 'pacmap', 'RAW'],
                pre_ranking: bool = True,
                n_components = 6,
                pipe=None,
                pretrained=None):
    '''
        in:
            df: dataframe with measurements
            clusterer: clusterer to use
            embedder: embedder to use
            pre_ranking: whether or not to use pre-ranking
            n_components: number of components to use for embedding
            n_clusters: number of clusters to use for clustering
        returns fitted pipeline
    '''
    
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    elif scaler == 'power':
        scaler = PowerTransformer()
        
    ######################################################    
    
    if (pipe is not None) & (pretrained=='clusterer'):
        clusterer = PreTrainedTransformer(pipe.named_steps['clusterer'])
    else:
        if clusterer == 'optics':
            clusterer = OPTICS(min_samples=15, metric='canberra', n_jobs=-1)
        elif clusterer == 'hdbscan':
            clusterer = HDBSCAN(min_cluster_size=5, min_samples=15, metric='canberra', cluster_selection_method='leaf')
        else:
            raise ValueError('Only optics and hdbscan are supported for density clustering')

    ######################################################

    if (pipe is not None) & (pretrained=='embedder'):
        embedder = PreTrainedTransformer(pipe.named_steps['embedder'])
    else:
        if embedder == 'pca':
            embedder = PCA(n_components=n_components, whiten=False)
        elif embedder == 'umap':
            embedder = umap.UMAP(n_components=n_components, n_neighbors=500, n_epochs=500,
                                metric='manhattan', init='spectral',
                                random_state=123, densmap=False, min_dist=0)
        elif embedder == 'trimap':
            pipeline_emb = clusterer.fit(
                            trimap.TRIMAP(n_dims=n_components, n_inliers=12, n_outliers=4, 
                                        distance='manhattan', n_iters=500, lr=0.09).fit_transform(
                                scaler.fit_transform(df)
                                )
                            )
            return pipeline_emb
        elif embedder == 'pacmap':
            embedder = pacmap.PaCMAP(n_components=n_components, 
                                        n_neighbors=500, num_iters=500, lr=0.9, 
                                        apply_pca=False)
        elif embedder == 'RAW':
            embedder = None
    
    ######################################################
    
    if pre_ranking:
        scaler = None
        ranker = Ranker
    
    le_pipe_list = [('scaler', scaler)]
    le_pipe_list += [('ranker', ranker)] if pre_ranking else []
    le_pipe_list += [('embedder', embedder)]
    le_pipe_list += [('clusterer', clusterer)]    
    le_pipe = Pipeline(le_pipe_list)
    le_pipe.fit(df)
    
    return le_pipe


In [ ]:
from collections import defaultdict

def enumerate_step(xs, start=0, step=1):
    for x in xs:
        yield (start, x)
        start += step

### Plot N components versus NMI/ARI with k-means

In [ ]:
pre_ranking = False
embedder = 'umap'
clusterer = 'kmeans'
scaler = 'standard'
n_comp_range = range(2,32,1)
n_clust_range = range(2,20,4)

set_dict = {
    'pre_ranking': pre_ranking,
    'embedder': embedder,
    'clusterer': clusterer,
    'scaler': scaler    
}

res_dict = defaultdict(dict)
raw_dict = {}
pipe_res = None

for n_clust in n_clust_range:
    raw_dict[n_clust] = pipe_fitter_fixed(df_, n_components=None, n_clusters=n_clust,
                                        pre_ranking=pre_ranking, embedder=None,
                                        clusterer=clusterer, scaler=scaler, cluster_init=None)
    
for n_comp in tqdm(n_comp_range):    
    cluster_centers = None
    for n_clust in n_clust_range:        
        pipe_res = pipe_fitter_fixed(df_, n_components=n_comp, n_clusters=n_clust,
                            pre_ranking=pre_ranking, embedder=embedder, 
                            clusterer=clusterer, scaler=scaler, pipe=pipe_res, 
                            pretrained='embedder', cluster_init=cluster_centers)        
        res_dict[(n_comp, n_clust)]['pipe'] = pipe_res
        #cluster_centers = pipe_res.named_steps['clusterer'].cluster_centers_
        #cluster_centers = np.vstack([cluster_centers, 
        #                            np.random.random((4,n_comp))])
        
    pipe_res = None

In [ ]:
# for HDBSCAN only (cluster.labels_>=0)
for key, value in res_dict.items():
    emb_labels = value['pipe']['clusterer'].labels_
    full_labels = raw_dict[key[1]]['clusterer'].labels_
    
    res_dict[key]['ARI'] = ARI(emb_labels, 
                               full_labels)
    res_dict[key]['NMI'] = NMI(emb_labels, 
                               full_labels) 

In [ ]:
plot_df = pd.DataFrame([{'n_comp': k[0], 
                         'n_cluster': k[1], 
                         'ARI': v['ARI'], 
                         'NMI': v['NMI']} 
                        for k,v in res_dict.items()])
plot_df = pd.melt(plot_df, id_vars=['n_comp', 'n_cluster'], value_vars=['ARI', 'NMI'], var_name='metric', value_name='score')

In [ ]:
cm = plt.get_cmap('gist_rainbow')

sns.relplot(data=plot_df.query('~n_cluster.isin([]) & n_comp<33'), 
            x='n_comp', y='score', col='metric', hue='n_cluster', kind='line') #  palette=cm

plt.ylim(0.21,1)
plt.suptitle(f'{embedder} + {clusterer}. Sample size: {sample_size}.')
plt.savefig(f'./../artifacts/{embedder}_{clusterer}_{sample_size}.eps', dpi=300, bbox_inches='tight')

fname = "__".join([f"{k}_{str(v)}" for k,v in set_dict.items()]+[f'sample_size_{sample_size}'])
plot_df.to_csv(f'./../artifacts/ClusterAlignment_{fname}.csv', index=False)

### Plot N components versus number of retrieved HDBSCAN clusters

In [ ]:
pre_ranking = False
embedder = 'pacmap'
clusterer = 'hdbscan'
scaler = 'standard'
n_comp_range = range(2,64,1)

set_dict = {
    'pre_ranking': pre_ranking,
    'embedder': embedder,
    'clusterer': clusterer,
    'scaler': scaler    
}

res_dict = defaultdict(dict)
raw_dict = {}
pipe_res = None


for n_comp in tqdm(n_comp_range):        
    pipe_res = pipe_fitter_density(df_, n_components=n_comp,
                            pre_ranking=pre_ranking, embedder=embedder, 
                            clusterer=clusterer, scaler=scaler, pipe=None,
                            pretrained='embedder')        
    res_dict[n_comp]['pipe'] = pipe_res
    pipe_res = None
    
raw_density = pipe_fitter_density(df_, n_components=n_comp,
                                        pre_ranking=pre_ranking, embedder=None,
                                        clusterer=clusterer, scaler=scaler)   

In [ ]:
num_outliers = np.bincount(raw_density.named_steps['clusterer'].labels_+1)[0]

for comp, res in res_dict.items():
    # scores like before using all clusters
    if embedder=='trimap':
        _res = res['pipe']
    else:
        _res = res['pipe']['clusterer']
        
    res_dict[comp]['ARI_full'] = ARI(_res.labels_, raw_density['clusterer'].labels_)
    res_dict[comp]['NMI_full'] = NMI(_res.labels_, raw_density['clusterer'].labels_)
    
    # scores based on the non-outlier clusters only, as indicated by the raw_density results with -1
    non_outlier_mask = raw_density['clusterer'].labels_>=0
    res_dict[comp]['ARI_non_outlier'] = ARI(_res.labels_[non_outlier_mask], 
                                            raw_density['clusterer'].labels_[non_outlier_mask])
    res_dict[comp]['NMI_non_outlier'] = NMI(_res.labels_[non_outlier_mask], 
                                            raw_density['clusterer'].labels_[non_outlier_mask])

    # number of outliers
    res_dict[comp]['num_outliers'] = sum(_res.labels_>=0)
    
    # number of clusters
    res_dict[comp]['num_clusters'] = len(np.unique(_res.labels_))
    

In [ ]:
plot_df = pd.DataFrame([{'n_comp': k, 
                         'n_outliers': v['num_outliers'],
                         'n_clusters': v['num_clusters'],
                         'ARI_full': v['ARI_full'], 
                         'NMI_full': v['NMI_full'],
                         'ARI_non_outlier': v['ARI_non_outlier'],
                         'NMI_non_outlier': v['NMI_non_outlier']                         
                         } 
                        for k,v in res_dict.items()])
plot_df = pd.melt(plot_df, 
                  id_vars=['n_comp', 'n_clusters'],
                  value_vars=['ARI_full', 'NMI_full', 'ARI_non_outlier', 'NMI_non_outlier', 'n_outliers'], 
                  var_name='metric', value_name='score')

In [ ]:
sns.lineplot(data=plot_df.query('~n_clusters.isin([]) & n_comp<64 & metric!="n_outliers"'), 
            x='n_comp', y='score', hue='metric', palette='tab10') #  

plt.suptitle(f'{embedder} + {clusterer}. Sample size: {sample_size}.')
plt.savefig(f'./../artifacts/{embedder}_{clusterer}_{sample_size}.eps', dpi=300, bbox_inches='tight')

fname = "__".join([f"{k}_{str(v)}" for k,v in set_dict.items()]+[f'sample_size_{sample_size}'])
plot_df.to_csv(f'./../artifacts/ClusterHallucination_{fname}.csv', index=False)

In [ ]:
pca_res = pd.read_csv('./../artifacts/ClusterHallucination_pre_ranking_False__embedder_pca__clusterer_hdbscan__scaler_standard__sample_size_25000.csv')
umap_res = pd.read_csv('./../artifacts/ClusterHallucination_pre_ranking_False__embedder_umap__clusterer_hdbscan__scaler_standard__sample_size_25000.csv')
trimap_res = pd.read_csv('./../artifacts/ClusterHallucination_pre_ranking_False__embedder_trimap__clusterer_hdbscan__scaler_standard__sample_size_25000.csv')
pacmap_res = pd.read_csv('./../artifacts/ClusterHallucination_pre_ranking_False__embedder_pacmap__clusterer_hdbscan__scaler_standard__sample_size_25000.csv')
plot_df = pd.concat([pca_res.assign(embedder='pca'), 
                     umap_res.assign(embedder='umap'),
                     trimap_res.assign(embedder='trimap'),
                     pacmap_res.assign(embedder='pacmap')])

In [ ]:
raw_num_clusters = np.unique(raw_density.named_steps['clusterer'].labels_).shape[0]

In [ ]:
raw_num_clusters = 11
sns.lineplot(data=plot_df.query('~n_clusters.isin([]) & n_comp<64'), 
            x='n_comp', y='n_clusters', hue='embedder', palette='tab10')
plt.axhline(raw_num_clusters, ls='--', color='k', label='raw')
plt.xlabel("Number of components")
plt.ylabel("Number of clusters")
plt.savefig(f'./../artifacts/cluster_hallucination.eps', dpi=300, bbox_inches='tight')


In [ ]:
#Xpca = pipeline_pca.named_steps['pca'].transform(df.loc[:, meas_columns])
#Xemb = pipeline_emb.named_steps['embedder'].embedding_
#Xorg = df.loc[:, meas_columns].values
#perfClass = CDEmbeddingPerformance(n_neighbours=500, metric='manhattan', dcor_level=1)

#rnd_sel = np.random.randint(0,100_000, 4000)
#plt.plot(perfClass._return_dynamic_distance_correlation(Xpca[rnd_sel], Xorg[rnd_sel], n_bins=80))
#plt.plot(perfClass._return_dynamic_distance_correlation(Xemb[rnd_sel], Xorg[rnd_sel], n_bins=80))
#plt.legend(['pca', 'emb'])

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(20, 12)) 

ordering = pipeline_emb.named_steps['clusterer'].ordering_
reach = pipeline_emb.named_steps['clusterer'].reachability_[ordering]
labels = pipeline_emb.named_steps['clusterer'].labels_[ordering]
space = np.arange(labels.shape[0])
num_categories = len(np.unique(labels))
colormap = cm.get_cmap('viridis', num_categories)
colors = [colormap(i) for i in range(num_categories)]

for klass, color in zip(range(0, num_categories), colors):
    Xk = space[labels == klass]
    Rk = reach[labels == klass]
    ax[0].plot(Xk, Rk, color, alpha=0.8)
ax[0].plot(space[labels == -1], reach[labels == -1], "k.", alpha=0.3)
#ax[0].plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
#ax[0].plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax[0].set_ylabel("Reachability (epsilon distance)")
ax[0].set_xlim(1,15000)


############################################################################################################
############################################################################################################
############################################################################################################

ordering = pipeline_full.named_steps['clusterer'].ordering_
reach = pipeline_full.named_steps['clusterer'].reachability_[ordering]
labels = pipeline_full.named_steps['clusterer'].labels_[ordering]
space = np.arange(labels.shape[0])
num_categories = len(np.unique(labels))
colormap = cm.get_cmap('viridis', num_categories)
colors = [colormap(i) for i in range(num_categories)]

for klass, color in zip(range(0, num_categories), colors):
    Xk = space[labels == klass]
    Rk = reach[labels == klass]
    ax[1].plot(Xk, Rk, color, alpha=0.3)
ax[1].plot(space[labels == -1], reach[labels == -1], "k.", alpha=0.3)
#ax[1].plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
#ax[1].plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax[1].set_ylabel("Reachability (epsilon distance)")
ax[1].set_xlim(1,15000)
ax[1].set_ylim(20,40)

fig.suptitle("Reachability Plot")


## Bootstrapped Clustering

In [ ]:
gc.collect()
bclusters = clustering_bootstrapping(celldyn_full[meas_columns], n_clusters=10, n_bootstraps=60, sample_size=10000, clusterer='kmeans', n_init=1, align=True)
gc.collect()

In [ ]:
test_set = celldyn_full.sample(10000, replace=True)[meas_columns]
labels, centers, sizes = assign_clusters(test_set, bclusters, clusterer='kmeans')

In [ ]:
from sklearn.metrics.cluster import fowlkes_mallows_score, homogeneity_score, homogeneity_completeness_v_measure
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score, adjusted_mutual_info_score, pair_confusion_matrix

cluster_overlap_list = []
for i in range(labels.shape[0]):
    for j in range(i+1, labels.shape[0]):
        cluster_overlap_list.append(fowlkes_mallows_score(labels[i], labels[j]))

plt.hist(cluster_overlap_list, bins=30);

In [ ]:
# we want to re-order the cluster labels for each model such that the ordinal similarity between the models is maximized
init_sim = np.sum(np.corrcoef(labels))/labels.shape[0]**2
print(f"Initial similarity: {init_sim}")

#  we want to align the cluster id's based on the cluster centers or the cluster assignmnents
def align_cluster_assignments(labels: np.ndarray, centers: np.ndarray, method: str='centerwise', base_sel: int=0)-> np.ndarray:
    '''
        Centerwise, given, 1..N models, we align the 2,N models based on the cluster centers of model 1.
        I.e. we find the cluster center in model 2 that is closest to the cluster center in model 1 and assign the cluster id's accordingly
    '''
    if method == 'centerwise':
        base_centers = centers[base_sel]
        # find the closest cluster center for each model
        closest_centers = [[np.argmin([distance(bv.ravel(), _cv.ravel(), metric='cosine') 
                                            for _cv in cv]) 
                                                for bv in base_centers] 
                                                    for cv in centers[1:]
                                                    ]
        # now we have the closest cluster centers for each model, we can re-order the labels
        aligned_labels = [np.array([closest_centers[i][l] for l in labels[base_sel]]) for i in range(len(closest_centers))]
        aligned_centers = [centers[i][closest_centers[i]] for i in range(len(closest_centers))]

    '''
        TODO:
    	We want to re-order the cluster labels for each model such that the ordinal similarity between the models is maximized.
        We express this similarity with MCC (Matthews correlation coefficient)
    '''
    # first order the labels from the first model, use the order index
    # to order the labels of the other models

    # now change the labels of the other models to match the order of the first model


    return np.vstack(aligned_labels), np.stack(aligned_centers), closest_centers

aligned_labels, aligned_centers, closest_centers = align_cluster_assignments(labels, centers, method='centerwise', base_sel=0)
print(f"Aligned result: {np.sum(np.corrcoef(aligned_labels))/aligned_labels.shape[0]**2}")

In [ ]:
cluster_assignment = pd.DataFrame(aligned_labels.T, columns=['cluster_assignment_'+str(i) for i in range(aligned_labels.shape[0])])
cluster_assignment.index = test_set.index
cluster_assignment = cluster_assignment.assign(cluster_entropy=cluster_assignment.apply(lambda x: entropy(np.histogram(x, bins=10, density=True)[0]), axis=1))
cluster_assignment = cluster_assignment.assign(median_cluster=cluster_assignment.apply(lambda x: int(np.median(x)), axis=1))

In [ ]:
plot_df = celldyn_emb.loc[cluster_assignment.index].join(cluster_assignment[['cluster_entropy', 'median_cluster']], how='inner')
sns.scatterplot(data=plot_df, x='dim_2', y='dim_3', hue='median_cluster', palette='viridis', alpha=0.5, legend=True)

In [ ]:
cluster_num = 0
for plot_count in range(100):
    _v1, _v2 = random.sample(range(0, aligned_centers.shape[2]), 2)
    if _v1 != _v2:
        v1 = aligned_centers[:, cluster_num, _v1]
        v2 = aligned_centers[:, cluster_num, _v2]
        plt.scatter(v1, v2, c='r', alpha=0.05)
plt.title(f"Cluster-center spread for cluster {cluster_num}")

## NMI and ARI

## Fuzzy c-means

In [ ]:
fpcs = []

df =celldyn_full[meas_columns] # ['age',  'gender']]
#df = celldyn_emb[['dim_1', 'dim_2', 'dim_3', 'dim_4',  'dim_5', 'dim_6']]
n_bootstrap = 30
sample_count = 50000
for n in tqdm(range(n_bootstrap)):
    data = df.sample(sample_count)
    _idx = data.index
    
    _fpcs = []
    for ncenters, ax in enumerate(range(10),2):
        cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data.T.values, c=ncenters, m=1.1, 
                                        error=0.005, maxiter=1000, init=None, seed=123)

        # Store fpc values for later
        _fpcs.append(fpc)
    fpcs.append(_fpcs)

for i in range(n_bootstrap):
    plt.plot(np.arange(2, 12), fpcs[i], alpha=0.1, c='b')

plt.xlabel("Number of centers")
plt.ylabel("Fuzzy partition coefficient")

## GMM evaluation

* use ```model.aic``` and ```model.bic``` to evaluate GMM
* use ```model.score_samples``` to evaluate BGMM

In [ ]:
# get sample data
res_list = []
n_bootstrap = 10
sample_count = 5000
n_clusters = 20

df =celldyn_full[meas_columns] 
#df = celldyn_emb[['dim_1', 'dim_2', 'dim_3', 'dim_4',  'dim_5', 'dim_6', 'age', 'sex']]
cluster_sizes = []
for b in range(n_bootstrap):
    data = df.sample(sample_count)
    # extract the cluster centers and aic, bic using GMM for 1..10 clusters
    
    for n in range(2, n_clusters+1):
        gmm = GMM(n_components=n, covariance_type='full', random_state=123, max_iter=1000).fit(data)
        labels = gmm.predict(data)
        cluster_sizes.append(np.bincount(labels))

        #centers = gmm.means_
        aic = gmm.aic(data)
        bic = gmm.bic(data)
        silhouette_avg = silhouette_score(data, labels)
        davies_bouldin = davies_bouldin_score(data.values, labels)
        calinski_harabasz = calinski_harabasz_score(data.values, labels)
        mean_log_likelihood = gmm.score(data)

        # only keep the samples that are assigned to clusters with more than 50 samples
        # this is to avoid the problem of having too few samples in a cluster
        # and thus the cluster classification f1 score is not reliable

        f1, f1_std = cluster_classification_f1(data.values, labels)
        res_list.append({'bootstrap_round': b, 
                         'num_cluster': n, 
                         'AIC': aic, 
                         'BIC': bic, 
                         'silhouette_score': silhouette_avg,
                         'davies_bouldin_score': davies_bouldin,
                         'calinski_harabasz_score': calinski_harabasz,
                         'score': mean_log_likelihood,
                         'f1': f1,
                         'f1_std': f1_std})
res_df = pd.DataFrame(res_list)

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(18, 25))

sns.lineplot(data=res_df, x='num_cluster', y='AIC', alpha=0.1, c='b',  ax=ax[0,0])
sns.lineplot(data=res_df, x='num_cluster', y='BIC', alpha=0.1, c='b',  ax=ax[0,1])

sns.lineplot(data=res_df, x='num_cluster', y='silhouette_score', alpha=0.1, c='b',  ax=ax[1,0])
sns.lineplot(data=res_df, x='num_cluster', y='davies_bouldin_score', alpha=0.1, c='b',  ax=ax[1,1])

sns.lineplot(data=res_df, x='num_cluster', y='calinski_harabasz_score', alpha=0.1, c='b',  ax=ax[2,0])
sns.lineplot(data=res_df, x='num_cluster', y='score', alpha=0.1, c='b',  ax=ax[2,1])

sns.lineplot(data=res_df, x='num_cluster', y='f1', alpha=0.1, c='b',  ax=ax[3,0])
sns.lineplot(data=res_df, x='num_cluster', y='f1_std', alpha=0.1, c='b',  ax=ax[3,1])

fig.suptitle("GMM clustering, bootstrap N=10, sample size=5000")
ax[0,0].set_title("AIC")
ax[0,1].set_title("BIC")

ax[1,0].set_title("Silhouette score")
ax[1,1].set_title("David Bouldin score")

ax[2,0].set_title("Calinski Harabasz score")
ax[2,1].set_title("Mean log likelihood")

ax[3,0].set_title("F1 score")
ax[3,1].set_title("F1 score std")


In [ ]:
res_df

In [ ]:
# get sample data
res_list = []
n_bootstrap = 20
sample_count = 10000
n_clusters = 20

df =celldyn_full[meas_columns] 
#df = celldyn_emb[['dim_1', 'dim_2', 'dim_3', 'dim_4',  'dim_5', 'dim_6', 'age', 'sex']]
for b in tqdm(range(n_bootstrap)):
    data = df.sample(sample_count)
    # extract the cluster centers and aic, bic using GMM for 1..10 clusters    
    
    gmm = BGMM(n_components=n_clusters, covariance_type='full', random_state=123, max_iter=1000).fit(data)
    #labels = gmm.predict(data)
    #centers = gmm.means_
    res_list.append({'bootstrap_round': b, 
                     'mean_log_likelihood': gmm.score(data)})
res_df = pd.DataFrame(res_list)

In [ ]:
df =celldyn_full[meas_columns] 
data = df.sample(1000)
n_comp = 3
gmm = BGMM(n_components=n_comp, covariance_type='full', random_state=123, max_iter=1000).fit(data)
bgmm_res = gmm.predict_proba(data)
AIC = gmm.aic(data)
BIC = gmm.bic(data)
data['log_likelihood_gmm'] = gmm.score_samples(data)
data[[f'cluster_{i}' for i in range(n_comp)]] = bgmm_res
res_df = pd.DataFrame(data=bgmm_res, columns=[f'cluster_{i}' for i in range(n_comp)])
res_df.index=data.index


In [ ]:
data[[f'cluster_{i}' for i in range(n_comp)]].mean()

In [ ]:
AIC, BIC

In [ ]:
clust = 'cluster_2'
plot_df = celldyn_emb.loc[data.index].join(data[['log_likelihood_gmm', clust]], how='inner')
sns.scatterplot(data=plot_df[plot_df.log_likelihood_gmm>=0], x='dim_1', y='dim_3', hue=clust, palette='viridis', alpha=0.5, legend=True)
sns.scatterplot(data=plot_df[plot_df.log_likelihood_gmm<0], x='dim_1', y='dim_3', color='black', alpha=0.1, legend=True)
